# Land market analysis

Import modules:

In [42]:
import pandas as pd
import time
import os
import json
from pprint import pprint
import requests
from datetime import datetime

Fetch data from IMX API:

In [60]:
API_URL = "https://api.sandbox.immutable.com/v3/orders"

def fetch_orders():
    all_orders = []
    params = {
        "status": "active",  # Only fetch orders that are currently active. Set to "filled" to fetch past orders.
        "page_size": 200,  # Request the maximum number of items per page
    }

    while True:
        print(f"{datetime.now()} Fetching next batch...")
        response = requests.get(API_URL, params=params)
        if response.status_code != 200:
            print("Failed to fetch data: HTTP Status Code", response.status_code)
            break

        data = response.json()
        # Filter to ensure orders are not filled (amount_sold is None) and match the collection name "Illuvium Land"
        active_orders = [
            order for order in data["result"]
            if order.get("amount_sold") is None and
               order["sell"]["data"]["properties"]["collection"]["name"] == "Illuvium Land"
        ]
        all_orders.extend(active_orders)
        
        # Check if a cursor is provided to fetch the next page
        cursor = data.get("cursor")
        break
        if cursor:
            params["cursor"] = cursor
        else:
            break

    return all_orders

data = fetch_orders()

2024-06-25 21:26:29.845475 Fetching next batch...


In [22]:
# Load data from JSON file
def load_json(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)
    return data

data = load_json('./data/imx/active_orders.json')
pprint(data[0])

{'amount_sold': None,
 'buy': {'data': {'decimals': 18,
                  'quantity': '2700000000000000000',
                  'quantity_with_fees': '2835000000000000000',
                  'token_address': ''},
         'type': 'ETH'},
 'expiration_timestamp': '2123-06-23T19:00:00Z',
 'maker_fees': {'decimals': 18,
                'fees': [],
                'quantity_with_fees': '2700000000000000000',
                'symbol': 'ETH',
                'token_type': 'ETH'},
 'maker_taker_type': 'maker',
 'order_id': 624044,
 'sell': {'data': {'id': '0xf9a62af1d4b39892f2c138904160656a64f3b6a5ac106106ff2f06c21adc7e84',
                   'properties': {'collection': {'icon_url': 'https://assets.sepolia-illuvium-game.io/web/static/collection/ingots/ingots_marketplace_icon.webp',
                                                 'name': 'Illuvium Ingots'},
                                  'image_url': 'https://sepolia.media.illuvium.io/web/NFT/Ingots/TungarIngot/TungarIngot_default_default_

Preprocess data:

In [59]:
for order in data:
    collection_name = order['sell']['data']['properties']['collection']['name']
    print(collection_name)
